In [1]:
import math
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from pathlib import Path  # pathlib.Path formats pathnames to suit your operating system

# "notebook" plotting mode allows for interactive (zoom-able) plots, "inline" doesn't
%matplotlib notebook   
# %matplotlib inline

#function to load the datafile
# find and parse data into baseline , null, TE , and Polarized
def load(filename = 'test_data.csv'):
    return pd.read_csv(filename, index_col=[0])
    
def clean_na(dirtydf):
    #remove any rows with ---, null readings
    #check to see if something went wrong and value is nonetype
    print(dirtydf is None)
    cleandf = dirtydf.drop(dirtydf[dirtydf["NMR Status"]=='---'].index)
    return cleandf

def seperateBL(cleandf):
    #add values to different sets based on NMR status column
    BL_df = pd.DataFrame(cleandf.loc[cleandf['NMR Status']=='Baseline'])
    return BL_df

def seperateTE(cleandf):
    TE_df = pd.DataFrame(cleandf.loc[cleandf['NMR Status']=='TE'])
    return TE_df

def seperatePL(cleandf):
    PL_df = pd.DataFrame(cleandf.loc[cleandf['NMR Status']=='Polarization'])
    return PL_df

dirtydata = load()
no_null_data = clean_na(dirtydata)
baselines = seperateBL(no_null_data)
thermal = seperateTE(no_null_data)
polarized = seperatePL(no_null_data)

            
#create series of frequency values for use on x axis
#center freq of 32.925 MHz, span of .8 MHz, 401 values for NMR data per sweep
#32.125-33.725, increments of .00399 MHz will correspond to each new data point
#print(len(df.columns)) to find # columns

#Have 83 columns of BS in front of NMR data, can generalize by subtracting that from len to find # of NMR readings
def freqs(any_df):
    #want to substract data columns from total, increment through frequency span in that many steps.
    columns = any_df.columns
    start = columns.get_loc('NMR Data')
    data_points = len(columns) - start
    freq_list = []
    count = 0
    increment = 1.6/data_points
    while count <=data_points:
        freq_list.append((count*increment)+32.125)
        count+=1
    #input a df, output a list(so we can zip w data) with the increments of frequency hit in MHz
    #freq_series = pd.Series(freq_list)
    return freq_list
    #print(len(freq_list))

#TEST FREQS
NMR_freqs_BL = freqs(baselines)
#print(NMR_freqs)

def average_of_sweeps(any_df):
    #want to avg over all the data in a column for a sub-df (BL,TE etc), and make one series of data to plug into plot
    #Will make this function so it can be used for any sub-df
    data_cols = len(any_df.columns)
    columns = any_df.columns
    count = columns.get_loc('NMR Data') #should be start
    data_list = []
    while count <= data_cols:
        
        single_col = any_df.iloc[:,count:] # this selects one columns and means it
        mean = single_col.mean()
        data_list.append(mean)
        count+=1
    data_series = pd.Series(data_list)
    return data_series

NMR_data_BL = average_of_sweeps(baselines)
print(NMR_data_BL[0:5])

                    
def not_zip_lol(freqs,data):
    #make empty data frame, add lists to it one by one 
    final_df = pd.DataFrame()
    final_df["Frequency"]=freqs
    final_df["Voltage"] = data
    return final_df
    


def zip_it_up(freqs,datas):
    #zip the data and freq lists, make into a dataframe
    combined_list = list(zip(freqs,datas))
    final_df = pd.DataFrame(combined_list, columns=['Frequency','Voltage'])
    return final_df

#DELETE ZIP 
#TEST ZIP
#final_df_BL = zip_it_up(NMR_freqs_BL,NMR_data_BL)
#print(final_df_BL)

##Questions:
#how to use zip ---->> DONT REALLY HAVE TO
#What is the column name for frequency MHz? --------> use centroid and span to find freqs
#how to return df from function inputed file so i can do more stuff to it? --------> IM A IDIOT
#should i split these functions and act them all seperately? yes i should, -------->of fucking course

#Need to figure out how to index columns_______> done
#Need to figure out how to average the like types, maybe not though --------------> need to figure out that function

#DO i need to average columns?
#{Unnamed BS} -----should be avoided with new average structure
#Threshold values for cleaning data, what var and what quantity? ________>dont worry

##Use enumerate to iterate list for index+value pairs per sweep (useful for indexing column names to numbers // Lists in general)
#probably want to transpose so 1 sweep is 1 column
#Use numpy arange to do span, total span is defined on a per sweep basis by "Freq Span (MHz)" and "Sweep Centroid"
#use numpy array to add arrays together element-wise
#Use dictionary to match keys(column header names) to values.
#Open file, enumerate through every line. Use the first line to assign interesting column names(define all earlier) to index for, then continue.
#then loop thru rows/lines of data to aggregate for each sweep , average NMR data by making a list of data frames

#scatter plots made using kind argument, specify plot type and axises
#df.plot(kind = 'scatter', x='Frequency',y='Voltage') ####DOnT USE THIS. USE MATPLOTLIB
#plt.show()

False
0    NMR Data        3.158093
Unnamed: 84     3.123...
1    Unnamed: 84     3.123810
Unnamed: 85     3.065...
2    Unnamed: 85     3.065233
Unnamed: 86     3.005...
3    Unnamed: 86     3.005886
Unnamed: 87     2.946...
4    Unnamed: 87     2.946986
Unnamed: 88     2.883...
dtype: object
